# Project 4: Lid Driven Cavity


- Resource: https://github.com/davidlin001/flow_sims/tree/master/SIMPLE


`Libraries`


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from solvers import *

`Initial Variables`


In [ ]:
Nx = Ny = 100
l = 1.0
nu = 0.01
Re = 1/nu
velocity = 1.0

dx = dy = l / (Nx - 1)

x = np.linspace(0.0, l, Nx)
y = np.linspace(0.0, l, Ny)
X,Y = np.meshgrid(x,y)

u = np.zeros((Nx+1, Ny))
v = np.zeros((Nx,Ny+1))
p = np.zeros((Nx, Ny))

`Boundry Conditions`


In [ ]:
def boundry(u):
	u[:, -1] = 1

In [ ]:
while True:
    u_star, d_u = x_momentum_solver(Nx,Ny, dx, dy, u, v, velocity,p)
    v_star,d_v = y_momentum_solver(Nx, Ny, dx, dy, u, v, velocity,p)
    p,p_prime = pressure_correction(Nx,Ny,dx,dy,u_star,v_star,p)
    
